# Project: Machine Learning


**Instructions for Students:**

Please carefully follow these steps to complete and submit your project:

1. **Completing the Project**: You are required to work on and complete all tasks in the provided project. Be disciplined and ensure that you thoroughly engage with each task.
2. **Creating a Google Drive Folder**: Each of you must create a new folder on your Google Drive if you haven't already. This will be the repository for all your completed assignment and project files, aiding you in keeping your work organized and accessible.
3. **Uploading Completed Project**: Upon completion of your project, make sure to upload all necessary files, involving codes, reports, and related documents into the created Google Drive folder. Save this link in the 'Student Identity' section and also provide it as the last parameter in the `submit` function that has been provided.
4. **Sharing Folder Link**: You're required to share the link to your project Google Drive folder. This is crucial for the submission and evaluation of your project.
5. **Setting Permission toPublic**: Please make sure your Google Drive folder is set to public. This allows your instructor to access your solutions and assess your work correctly.

Adhering to these procedures will facilitate a smooth project evaluation process for you and the reviewers.


## Student Identity


In [1]:
# @title #### Student Identity
student_id = "REAEAYH8" # @param {type:"string"}
name = "Aburizal Adi Nugroho" # @param {type:"string"}
drive_link = "https://drive.google.com/drive/folders/1pRIbA2Zphrtg_TS5N0ypHHTXkUBs50tI?usp=drive_link"  # @param {type:"string"}

assignment_id = "00_ml_project"

# Import grader package
!pip install rggrader
from rggrader import submit, submit_image

## Project Description

In this Machine Learning Project, you will create your own supervised Machine Learning (ML) model. We will use the full FIFA21 Dataset and we will identify players that are above average.

We will use the column "Overall" with a treshold of 75 to define players that are 'Valuable'. This will become our target output which we need for a supervised ML model. Because we use the "Overall" as our target output, you cannot use "Overall" in your features, this will be explained further below.

This project will provide a comprehensive overview of your abilities in machine learning, from understanding the problem, choosing the right model, training, and optimizing it.


## Grading Criteria

Your score will be awarded based on the following criteria:

- 100: The model has an accuracy of more than 80% and an F1 score of more than 85%. This model is excellent and demonstrates a strong understanding of the task.
- 90: The model has an accuracy of more than 75% and an F1 score of more than 80%. This model is very good, with some room for improvement.
- 80: The model has an accuracy of more than 70% and an F1 score between 70% and 80%. This model is fairly good but needs improvement in balancing precision and recall.
- 70: The model has an accuracy of more than 65% and an F1 score between 60% and 70%. This model is below average and needs significant improvement.
- 60 or below: The model has an accuracy of less than 65% or an F1 score of less than 60%, or the student did not submit the accuracy and F1 score. This model is poor and needs considerable improvement.


Rmember to make a copy of this notebook in your Google Drive and work in your own copy.

Happy modeling!

> Note: If you get the accuracy of 100% and F1 score of 100%, while it may earn you good grades, it's an indication of overfitting.


In [1]:
# Write any package/module installation that you need
# pip install goes here, this helps declutter your output below
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

## Load the dataset and clean it

In this task, you will prepare and load your dataset. You need to download the full FIFA 21 Dataset from the link here: [Kaggle FIFA Player Stats Database](https://www.kaggle.com/datasets/bryanb/fifa-player-stats-database?resource=download&select=FIFA21_official_data.csv).

> Note: Make sure you download FIFA 21 dataset.
>
> ![FIFA21 Dataset](https://storage.googleapis.com/rg-ai-bootcamp/projects/fifa21_dataset-min.png)

After you download the dataset, you will then import the dataset then you will clean the data. For example there may be some empty cell in the dataset which you need to fill. Maybe there are also data that you need to convert to numeric value for analysis. Identify the data that is incomplete and fix them.

In the code block below, you can use the comments to guide you on what to do.


In [2]:
# Write your preprocessing and data cleaning here
pd.set_option("display.max_rows", 1000)
pd.set_option('display.max_columns', 500)

def converting_to_int(data):
  if type(data) == str:
    value_str = data.replace("€", "")
    last_char = value_str[len(value_str) - 1]
    if last_char == "M" or last_char == "K":
      clean_value_str = value_str[:len(value_str) - 1]
      return int(float(clean_value_str) * (1000 if last_char == "K" else 1000000))
    else:
      return int(float(value_str))
  else:
    return data

def converting_currency_string_to_int(column):
  return column.apply(converting_to_int)

# Load your data
fifa_df = pd.read_csv("datasets/FIFA21_official_data.csv")

# Check your data for empty cell
# print(fifa_df.isnull().sum())

# Fill the empty cell with data or drop the column
fifa_df[["Value", "Wage", "Release Clause"]] = fifa_df[["Value", "Wage", "Release Clause"]].apply(converting_currency_string_to_int)
pd.set_option('display.max_columns', 500)
fifa_df.dropna(subset=["Agility", "Balance", "Jumping", "Vision", "SlidingTackle", "Composure", "DefensiveAwareness"], inplace=True)
fifa_df.drop(columns=["Marking", "Loaned From", "Joined", "Contract Valid Until", "Position", "Jersey Number", "Release Clause", "Club"], inplace=True)

In [3]:
fifa_df['OK Player'] = fifa_df['Overall'].apply(lambda x: 1 if x >= 75 else 0)

normal_val = fifa_df["Value"]
normal_val = (normal_val - np.mean(normal_val)) / (np.max(normal_val) - np.min(normal_val))
fifa_df["Value"] = normal_val

filtered_data = fifa_df[["Value", "Potential", "Crossing", "Finishing", "HeadingAccuracy", "ShortPassing", "Volleys", "Dribbling", "LongPassing", "BallControl", "Acceleration", "SprintSpeed", "Agility", "Reactions", "Balance", "ShotPower", "Jumping", "Stamina", "Strength", "LongShots", "Aggression", "Interceptions", "Positioning", "Vision", "Penalties", "Composure", "StandingTackle", "SlidingTackle", "GKDiving", "GKHandling", "GKKicking", "GKPositioning", "GKReflexes", "DefensiveAwareness", "Overall", "OK Player"]]

In [4]:
features = filtered_data.drop(columns=["Overall", "OK Player"])

X = features.values
y = filtered_data["OK Player"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

model = LogisticRegression(max_iter=100000)
model.fit(X_train, y_train)

coefficient = model.coef_     #w
intercept = model.intercept_  #b

decision_boundary = -intercept / coefficient
decision_boundary = decision_boundary[0][0]

In [5]:
def linear_function(w, x, b):
  return w * x + b

def sigmoid_function(z):
  return 1 / (1 + np.exp(-z))

eps = 1e-7 #
def compute_cost_logistic(X, y, w, b):
  m = X.shape[0]
  cost = 0.0
  for i in range(m):
    z_i = np.dot(X[i], w) + b
    f_wb_i = sigmoid_function(z_i)

    cost += -y[i]*np.log(f_wb_i + eps) - (1-y[i])*np.log(1-f_wb_i + eps)

  cost = cost / m
  return cost

cost = compute_cost_logistic(X_train, y_train, coefficient[0], intercept)

print(f"cost {cost}")
print(f"Accuracy on training set: {model.score(X_train, y_train)}")

cost [0.08921248]
Accuracy on training set: 0.9621094958243118


In [6]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

y_hat = model.predict(X_test)
y_hat

cm = confusion_matrix(y_test, y_hat)
print(cm)

print(f1_score(y_test, y_hat))
print(accuracy_score(y_test, y_hat))

[[2760   45]
 [  72  357]]
0.8592057761732852
0.963821892393321


## Build and Train your model

In this task you will analyze the data and select the features that is best at predicting if the Player is a 'Valuable' player or not.

The first step is to **define the target output** that you will use for training. Here's an example of how to create a target output:

- `df['OK Player'] = df['Overall'].apply(lambda x: 1 if x >= 50 else 0) #Define the OK Player using treshold of 50.`

Next you will **identify the features** that will best predict a 'Valuable' player. You are required to **submit the features you selected** in the Submission section below. Because we use the "Overall" as our target output, the use of "Overall" in your features is not allowed. You will automatically get 0 if you submit "Overall" in your features.

Once you identify the features, you will then **split the data** into Training set and Testing/Validation set.

Depending on the features you selected, **you may need to scale the features**.

Now you will **train your model, choose the algorithm** you are going to use carefully to make sure it gives the best result.

Once you have trained your model, you need to test the model effectiveness. **Make predictions against your Testing/Validation set** and evaluate your model. You are required to **submit the Accuracy Score and F1 score** in the Submission section below.

In the code block below, you can use the comments to guide you on what to do.

We have also provided 3 variables that you must use in your code, `ml_features`, `ml_accuracy` and `ml_f1_score`. You can move the variables around your code, assign values to them, but you cannot delete them.


In [10]:
# Write your code here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

pd.set_option("display.max_rows", 1000)
pd.set_option('display.max_columns', 500)

# Functions
def converting_to_int(data):
  if type(data) == str:
    value_str = data.replace("€", "")
    last_char = value_str[len(value_str) - 1]
    if last_char == "M" or last_char == "K":
      clean_value_str = value_str[:len(value_str) - 1]
      return int(float(clean_value_str) * (1000 if last_char == "K" else 1000000))
    else:
      return int(float(value_str))
  else:
    return data

def converting_currency_string_to_int(column):
  return column.apply(converting_to_int)

def linear_function(w, x, b):
  return w * x + b

def sigmoid_function(z):
  return 1 / (1 + np.exp(-z))

eps = 1e-7 #
def compute_cost_logistic(X, y, w, b):
  m = X.shape[0]
  cost = 0.0
  for i in range(m):
    z_i = np.dot(X[i], w) + b
    f_wb_i = sigmoid_function(z_i)

    cost += -y[i]*np.log(f_wb_i + eps) - (1-y[i])*np.log(1-f_wb_i + eps)

  cost = cost / m
  return cost

# Load your data
fifa_df = pd.read_csv("datasets/FIFA21_official_data.csv")

fifa_df[["Value", "Wage", "Release Clause"]] = fifa_df[["Value", "Wage", "Release Clause"]].apply(converting_currency_string_to_int)
pd.set_option('display.max_columns', 500)
fifa_df.dropna(subset=["Agility", "Balance", "Jumping", "Vision", "SlidingTackle", "Composure", "DefensiveAwareness"], inplace=True)
fifa_df.drop(columns=["Marking", "Loaned From", "Joined", "Contract Valid Until", "Position", "Jersey Number", "Release Clause", "Club"], inplace=True)

#Normalize Value Column
normal_val = fifa_df["Value"]
normal_val = (normal_val - np.mean(normal_val)) / (np.max(normal_val) - np.min(normal_val))
fifa_df["Value"] = normal_val

# Define the target output (Good >= 75)
fifa_df['OK Player'] = fifa_df['Overall'].apply(lambda x: 1 if x >= 75 else 0)

# Identify the features you will use in your model
ml_features = ["Value", "Potential", "Crossing", "Finishing", "HeadingAccuracy", "ShortPassing", "Volleys", "Dribbling", "LongPassing", "BallControl", "Acceleration", "SprintSpeed", "Agility", "Reactions", "Balance", "ShotPower", "Jumping", "Stamina", "Strength", "LongShots", "Aggression", "Interceptions", "Positioning", "Vision", "Penalties", "Composure", "StandingTackle", "SlidingTackle", "GKDiving", "GKHandling", "GKKicking", "GKPositioning", "GKReflexes", "DefensiveAwareness"]
features = fifa_df[ml_features]

# Split data into training set and test set
X = features.values
y = fifa_df["OK Player"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

# Scale the features (if needed, optional)

# Train the model
model = LogisticRegression(max_iter=100000)
model.fit(X_train, y_train)

coefficient = model.coef_     #w
intercept = model.intercept_  #b

cost = compute_cost_logistic(X_train, y_train, coefficient[0], intercept)

print(f"cost {cost}")
print(f"Accuracy on training set: {model.score(X_train, y_train)}")

# Make predictions using the test set
y_hat = model.predict(X_test)
y_hat

cm = confusion_matrix(y_test, y_hat)

# Evaluate the model
ml_accuracy = accuracy_score(y_test, y_hat)
ml_f1_score = f1_score(y_test, y_hat)

print(f"ml accuracy {ml_accuracy}")
print(f"ml f1 score {ml_f1_score}")

cost [0.08921248]
Accuracy on training set: 0.9621094958243118
ml accuracy 0.963821892393321
ml f1 score 0.8592057761732852


## Submission

Once you are satisfied with the performance of your model, then you run the code block below to submit your project.


In [9]:
# Submit Method

# Do not change the code below
question_id = "01_ml_project_features"
submit(student_id, name, assignment_id, str(ml_features), question_id, drive_link)
question_id = "02_ml_project_accuracy"
submit(student_id, name, assignment_id, str(ml_accuracy), question_id, drive_link)
question_id = "03_ml_project_f1score"
submit(student_id, name, assignment_id, str(ml_f1_score), question_id, drive_link)

'Assignment successfully submitted'

## FIN
